In [4]:
from __future__ import division, print_function
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
from glob import glob
import os
import re
import pickle
from collections import Counter, defaultdict
#pip install tqdm
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix

In [2]:
%load_ext watermark
%watermark -v -m -p numpy,scipy,pandas,matplotlib,statsmodels,sklearn -g

CPython 2.7.16
IPython 5.8.0

numpy 1.16.5
scipy 1.2.1
pandas 0.24.2
matplotlib 2.2.3
statsmodels 0.10.1
sklearn 0.20.3

compiler   : MSC v.1500 64 bit (AMD64)
system     : Windows
release    : 7
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 8
interpreter: 64bit
Git hash   :


In [3]:
# Поменяйте на свой путь к данным
PATH_TO_RAW_DATA = '../data/raw'
PATH_TO_INTERIM_DATA = '../data/interim'

In [104]:
class MarkovChainWithUnknownState():
    def __init__(self):
        self.init_probs = {}
        self.probs = {}
        self.unknown_probs = {}
        self.unknown_init_prob = 0
    
    def fit(self, X):
        #count bigrams in a shape of dict of dicts
        for sequence in X:
            for word1, word2 in zip(sequence[:-1], sequence[1:]):
                if word1 not in self.probs:
                    self.probs[word1] = {}
                if word2 not in self.probs[word1]:
                    self.probs[word1][word2] = 1
                else:
                    self.probs[word1][word2] += 1
        #for each word1
        #normalize counts into probabilities so that the probability of an unknown word2 were included and equal to 1/(total+1)
        totals = {word1: sum(self.probs[word1].values()) for word1 in self.probs}
        for word1 in self.probs:
            for word2 in self.probs[word1]:
                self.probs[word1][word2] /= (totals[word1] + 1)
            self.unknown_probs[word1] = 1 / (totals[word1] + 1)
        #set normalized init_probs and unknown_init_prob
        for word1 in totals:
            self.init_probs[word1] = totals[word1] / (sum(totals.values()) + sum(self.unknown_probs.values()))
        self.unknown_init_prob = sum(self.unknown_probs.values()) / (sum(totals.values()) + sum(self.unknown_probs.values()))
    
    def predict_proba(self, X):
        result = []
        for sequence in X:
            subresult = 0
            #probability of first word
            if sequence[0] not in self.init_probs:
                subresult += np.log(self.unknown_init_prob)
            else:
                subresult += np.log(self.init_probs[sequence[0]])
            #transition probabilities
            for word1, word2 in zip(sequence[:-1], sequence[1:]):
                if word1 not in self.probs:
                    #if word1 is unknown, probability word1 -> word2 is uniformly distributed 
                    #over known words (only word1 are counted) + one unknown word
                    subresult += np.log(1 / (len(self.probs) + 1))
                elif word2 not in self.probs[word1]:
                    #if word2 is unknown, get probability from self.unknown_probs
                    subresult += np.log(self.unknown_probs[word1])
                else:
                    subresult += np.log(self.probs[word1][word2])
            result.append(subresult)
        return result

In [105]:
def partition(lst, n, padding=None):
    if padding is not None and len(lst) % n > 0:
        padded_lst = lst + [padding] * (n - (len(lst) % n))
    else:
        padded_lst = lst
    return [padded_lst[idx : idx+n] for idx in range(0, len(padded_lst), n)]


def make_site_ID_freqs(path_to_csv_files):
    #read through data and count site frequencies
    site_freqs = Counter()
    file_pattern = os.path.join(path_to_csv_files, 'user[0-9]*.csv')
    for fpath in tqdm(sorted(glob(file_pattern)), 
                      desc='Calculating frequencies'):
        user_sites = pd.read_csv(fpath)['site'].values
        site_freqs.update(user_sites)
    #set site IDs from most to least frequent
    sorted_sites = [el[0] for el in sorted(site_freqs.items(), key=lambda x: -x[1])]
    site_ID_freqs = {site: (site_ID, site_freqs[site]) for site, site_ID in zip(sorted_sites, range(1, len(sorted_sites)+1))}
    return site_ID_freqs
    
def add_mc_probas(sessions_df, user_mcs):
    mc_proba_df = pd.DataFrame(index=sessions_df.index)
    for user_ID in tqdm(user_mcs, 
                        desc='Predicting probabilities of Markov chains on sessions'):
        mc_proba_df['mc{}_proba'.format(user_ID)] = user_mcs[user_ID].predict_proba(sessions_df.values)
    return pd.concat([sessions_df, mc_proba_df], axis=1)

def prepare_train_set(path_to_csv_files, session_length=10):
    site_ID_freqs = make_site_ID_freqs(path_to_csv_files)
    #read through data and encode sites with IDs
    user_site_IDs = {}
    user_ID_regex = re.compile('.*user([0-9]*).csv')
    file_pattern = os.path.join(path_to_csv_files, 'user[0-9]*.csv')
    user_mcs = {}
    for fpath in tqdm(sorted(glob(file_pattern)), 
                      desc='Encoding sites and training Markov chains'):
        user_ID = int(re.match(user_ID_regex, fpath).group(1))
        user_sites = pd.read_csv(fpath)['site'].values
        user_site_IDs[user_ID] = [site_ID_freqs[site][0] for site in user_sites]
        #train markov chain on site ID sequence
        user_mc = MarkovChainWithUnknownState()
        user_mc.fit([user_site_IDs[user_ID]])
        user_mcs[user_ID] = user_mc
    #split user site IDs into sessions
    sessions = []
    session_user_IDs = []
    for user_ID in tqdm(user_site_IDs, 
                        desc='Constructing sessions'):
        user_sessions = partition(user_site_IDs[user_ID], session_length, padding=0)
        sessions += user_sessions
        session_user_IDs += [user_ID] * len(user_sessions)
    sessions_df = pd.DataFrame(sessions, columns=['site{}'.format(i) for i in range(1, session_length+1)])
    sessions_df = add_mc_probas(sessions_df, user_mcs)
    sessions_df['user_id'] = session_user_IDs
    
    return sessions_df, site_ID_freqs

In [106]:
train_set = prepare_train_set(os.path.join(PATH_TO_RAW_DATA, '10users'))[0]
mc_cols = [col for col in train_set.columns if col.startswith('mc')]
user_pattern = re.compile('mc([0-9]*)_proba')
train_set['best_mc'] = train_set[mc_cols].apply(lambda row: int(re.match(user_pattern, row.idxmax()).group(1)), axis=1)
pred = (train_set['user_id'] == train_set['best_mc']).astype(int)
print('Score: ', sum(pred) / len(pred))
train_set[mc_cols + ['best_mc', 'user_id']]

SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdDYWxjdWxhdGluZyBmcmVxdWVuY2llcycsIG1heD0xMCwgc3R5bGU9UHJvZ3Jlc3NTdHlsZShkZXNjcmnigKY=


SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdFbmNvZGluZyBzaXRlcyBhbmQgdHJhaW5pbmcgTWFya292IGNoYWlucycsIG1heD0xMCwgc3R5bGU9UHLigKY=


SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdDb25zdHJ1Y3Rpbmcgc2Vzc2lvbnMnLCBtYXg9MTAsIHN0eWxlPVByb2dyZXNzU3R5bGUoZGVzY3JpcHTigKY=


SEJveChjaGlsZHJlbj0oSW50UHJvZ3Jlc3ModmFsdWU9MCwgZGVzY3JpcHRpb249dSdQcmVkaWN0aW5nIHByb2JhYmlsaXRpZXMgb2YgTWFya292IGNoYWlucyBvbiBzZXNzaW9ucycsIG1heD3igKY=



Score:  0.738496550743


,mc128_proba,mc33_proba,mc100_proba,mc39_proba,mc237_proba,mc207_proba,mc241_proba,mc50_proba,mc31_proba,mc127_proba,best_mc,user_id
0,-26.506005,-36.395912,-38.302682,-39.736039,-39.443509,-47.481420,-37.058816,-42.407541,-34.867139,-34.764745,128,128
1,-22.682230,-29.486649,-34.312930,-39.312455,-34.577663,-37.636743,-25.914881,-37.945626,-32.494330,-30.714101,128,128
2,-26.846511,-32.849942,-35.782801,-39.414673,-33.401345,-31.016092,-31.481631,-36.737864,-27.177589,-36.619800,128,128
3,-22.749369,-33.651744,-34.456095,-38.684177,-36.036720,-27.221778,-28.397006,-35.377666,-24.521930,-35.418637,128,128
4,-25.896664,-28.578684,-54.513256,-42.539204,-34.595183,-33.258504,-42.124813,-42.767101,-34.872432,-37.661076,128,128
5,-22.496323,-40.892971,-59.974189,-46.495233,-39.216204,-43.400270,-50.631623,-52.197156,-36.468288,-42.101909,128,128
6,-26.097135,-37.657564,-59.974189,-44.343903,-33.353759,-33.257889,-54.825597,-57.709240,-25.502250,-28.961653,31,128
7,-40.724862,-48.575575,-47.790928,-55.290434,-52.967542,-49.013022,-41.457396,-53.411947,-42.388798,-44.054015,128,128
8,-22.923986,-56.535730,-59.974189,-54.428215,-56.395161,-60.548137,-59.019572,-63.221324,-58.821563,-30.191940,128,128
9,-19.332499,-62.858071,-59.974189,-50.351112,-65.196389,-67.374359,-59.019572,-63.221324,-64.998470,-24.739973,128,128


In [72]:
user31_sites = pd.read_csv(os.path.join(PATH_TO_RAW_DATA, 
                                       '10users/user0031.csv'))['site'].values
user33_sites = pd.read_csv(os.path.join(PATH_TO_RAW_DATA, 
                                       '10users/user0033.csv'))['site'].values

In [73]:
mc31 = MarkovChainWithUnknown()
mc31.fit([user31_sites])

In [74]:
mc31.predict_proba([user31_sites[31:41], user33_sites[31:41]])

[-19.21140008628591, -16.816797275659557]

In [75]:
user31_sites[31:41]

array(['i1-js-14-3-01-11074-845302217-i.init.cedexis-radar.net',
       'b12.myspace.com', 'webmail.laposte.net', 'webmail.laposte.net',
       'webmail.laposte.net', 'webmail.laposte.net', 'trk.adbutter.net',
       'rs.mediapostcommunication.net', 'av.mediapostcommunication.net',
       'rbp.mxptint.net'], dtype=object)

In [76]:
user33_sites[31:41]

array(['www.google.fr', 'www.google.fr', 'www.google.fr',
       'fr.wikipedia.org', 'bits.wikimedia.org', 'meta.wikimedia.org',
       'login.wikimedia.org', 'bits.wikimedia.org', 'fr.wikipedia.org',
       'www.google.fr'], dtype=object)